In [11]:
import skyfield
from skyfield.api import EarthSatellite, load, wgs84, N, S, E, W
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, date
from pytz import timezone, all_timezones
import matplotlib.pyplot as plt

import requests
import os

import plotly.express as px

import sys
path = r'C:\Users\Virgil MESLE\Documents\0-Obsidian\Python\My_codes\Codes_CSUM\tools\main'
sys.path.append(path)
import functions_csum as fctcsum

##### Works only when connected to the internal network (wired)

In [12]:
## Charge le module 'temps'
ts = load.timescale()
now = datetime.now()
offset_days = 83
t0 = ts.from_datetime(timezone('UTC').localize(now) - timedelta(hours=1))
t1 = ts.from_datetime(timezone('UTC').localize(now + timedelta(days=offset_days)))

In [13]:
## get SAT from last tle
# last_TLE = fctcsum.get_and_save_TLE()

# last_TLE

In [14]:
# last_TLE = '1 98895U          24191.75347222  .00000000  00000-0  50000-4 0    05\n2 98895  62.0100 143.8636 0003415   0.0000   6.0370 14.96641939    07'
last_TLE = '1 00001U 00001A   24191.79513889 -.00000000  00000-0  00000-0 0    14\n2 00001  62.0000 146.2075 0002013 241.9600 344.4562 14.96474435000009'

In [15]:
satellite = fctcsum.TLE_to_sat(last_TLE)

In [16]:
# Hard coded CSUM Location & search parameters, to get from a .txt in the future
csum = wgs84.latlon(43.637 * N, 3.843 * E)
elev_min = 0

In [17]:
t, events = satellite.find_events(csum, t0, t1, altitude_degrees=elev_min)
event_names = [f'rise above {elev_min}', 'culminate', f'set below {elev_min}']
eph = load('de421.bsp')
sunlit = satellite.at(t).is_sunlit(eph)

In [18]:
## LOCAL 


for ti, event, sunlit_flag in zip(t, events, sunlit):
    name = event_names[event]
    if sunlit_flag : state = 'in sunlight'
    else : state = 'in shadow'  

    if name == 'culminate' :
        difference = satellite - csum
        topocentric = difference.at(ti)
        alt, az, distance = topocentric.altaz()
        print('{:25} {:15} {:15} {}'.format(
            ti.utc_strftime('%Y %b %d %H:%M:%S UTC'), name, state, alt
        ))
    else : 
        print('{:25} {:15} {:15}'.format(
            ti.utc_strftime('%Y %b %d %H:%M:%S UTC'), name, state
        ))

2024 Jul 09 14:58:09 UTC  rise above 0    in sunlight    
2024 Jul 09 15:03:48 UTC  culminate       in sunlight     17deg 36' 14.6"
2024 Jul 09 15:09:29 UTC  set below 0     in sunlight    
2024 Jul 09 16:36:14 UTC  rise above 0    in sunlight    
2024 Jul 09 16:42:41 UTC  culminate       in sunlight     56deg 42' 14.6"
2024 Jul 09 16:49:12 UTC  set below 0     in sunlight    
2024 Jul 09 18:17:27 UTC  rise above 0    in sunlight    
2024 Jul 09 18:22:52 UTC  culminate       in sunlight     12deg 59' 44.2"
2024 Jul 09 18:28:19 UTC  set below 0     in sunlight    
2024 Jul 09 19:59:52 UTC  rise above 0    in sunlight    
2024 Jul 09 20:04:09 UTC  culminate       in sunlight     05deg 58' 31.2"
2024 Jul 09 20:08:26 UTC  set below 0     in sunlight    
2024 Jul 09 21:40:28 UTC  rise above 0    in sunlight    
2024 Jul 09 21:45:34 UTC  culminate       in sunlight     10deg 08' 38.4"
2024 Jul 09 21:50:39 UTC  set below 0     in sunlight    
2024 Jul 09 23:19:40 UTC  rise above 0    in sunli